In [ ]:
from typing import List
import random


class Snake:
    def __init__(self, player_id):
        self.player_id = player_id
        self.body = []
        self.direction = (0, 0)
        self.score = 0
        self.food_eaten = 0

    def spawn(self):
        self.body = [(random.randint(5, 15), random.randint(5, 15))]
        self.direction = (1, 0)
        self.score = 0

    def move(self):

        head = self.body[0]
        new_head = (head[0] + self.direction[0], head[1] + self.direction[1])

        self.body.insert(0, new_head)
        if self.food_eaten > 0:
            self.food_eaten -= 1
        else:
            self.body.pop()



class Food:
    def __init__(self):
        self.position = (0, 0)
        self.spawn()

    def spawn(self):
        self.position = (random.randint(0, 19), random.randint(0, 19))



class MultiplayerSnakeGame:
    def __init__(self):
        self.width = 20
        self.height = 20
        self.snakes: List[Snake] = []
        self.foods: List[Food] = []
        self.map = [["" for _ in range(self.width)] for _ in range(self.height)]

    def add_snake(self, snake: Snake):
        self.snakes.append(snake)
        for x, y in snake.body:
            self.map[x][y] = snake.player_id

    def remove_snake(self, player_id: str):
        self.snakes = [snake for snake in self.snakes if snake.player_id != player_id]
        for x in range(self.width):
            for y in range(self.height):
                if self.map[x][y] == player_id:
                    self.map[x][y] = ""

    def add_food(self, food: Food):
        self.foods.append(food)
        self.map[food.position[0]][food.position[1]] = "food"

    def remove_food(self, position):
        self.foods = [food for food in self.foods if food.position != position]
        self.map[position[0]][position[1]] = ""

    def move_snakes(self):
        for snake in self.snakes:
            snake.move()

    def change_direction(self, player_id, direction):
        for snake in self.snakes:
            if snake.player_id == player_id:
                if (
                    snake.direction == (1, 0) and direction == (-1, 0)
                    or snake.direction == (-1, 0) and direction == (1, 0)
                    or snake.direction == (0, 1) and direction == (0, -1)
                    or snake.direction == (0, -1) and direction == (0, 1)
                ):
                    return
                snake.direction = direction

    def check_collisions(self):
        for snake in self.snakes:
            head = snake.body[0]
            if (
                head[0] < 0
                or head[0] >= self.width
                or head[1] < 0
                or head[1] >= self.height
            ):
                self.remove_snake(snake.player_id)
            elif (
                self.map[head[0]][head[1]] != "" and self.map[head[0]][head[1]] != "food"
            ):
                print("Collision")
                self.remove_snake(snake.player_id)
            elif head in [food.position for food in self.foods]:
                snake.score += 1
                snake.food_eaten += 1
                self.remove_food(head)

    def update_map(self):
        self.map = [["" for _ in range(self.width)] for _ in range(self.height)]
        for snake in self.snakes:
            for x, y in snake.body:
                self.map[x][y] = snake.player_id
        for food in self.foods:
            self.map[food.position[0]][food.position[1]] = "food"

    def spawn_food(self):
        if len(self.foods) >= 10:
            return
        new_food = Food()
        new_food.spawn()
        self.add_food(new_food)

    def spawn_snake(self, player_id):
        new_snake = Snake(player_id)
        new_snake.spawn()
        self.add_snake(new_snake)

    def update(self):
        self.move_snakes()
        self.check_collisions()
        self.update_map()
        self.spawn_food()


In [1]:
from IPython.display import display
from ipywidgets import Button, Label, GridspecLayout, VBox
import asyncio
import time

# Assume `MultiplayerSnakeGame` is already defined and initialized
game = MultiplayerSnakeGame()
game_active = True
FRAME_RATE = 0.5

def create_grid(data):
    rows = len(data)
    cols = len(data[0]) if rows else 0
    grid = GridspecLayout(rows, cols)
    for i in range(rows):
        for j in range(cols):
            grid[i, j] = Label(value=str(data[i][j]), layout={'border': '1px solid black'})
    return grid

def update_grid(grid, data):
    rows = len(data)
    cols = len(data[0]) if rows else 0
    for i in range(rows):
        for j in range(cols):
            grid[i, j].value = str(data[i][j])

# Create initial game map
map_widget = create_grid(game.map)

# Define and setup controls
up_button = Button(description="Up")
down_button = Button(description="Down")
left_button = Button(description="Left")
right_button = Button(description="Right")
start_button = Button(description="Start")

def start_game(b):
    print("Game started!")
    game.spawn_snake("player")
    global start_time
    start_time = time.time()  # Reset the start time when the game starts
    asyncio.create_task(game_loop())  # Start the game loop as an asynchronous task


up_button.on_click(lambda b: game.change_direction((0, -1)))
down_button.on_click(lambda b: game.change_direction((0, 1)))
left_button.on_click(lambda b: game.change_direction((-1, 0)))
right_button.on_click(lambda b: game.change_direction((1, 0)))
start_button.on_click(start_game)

import asyncio

async def game_loop():
    global game_active, start_time
    while game_active:
        elapsed_time = time.time() - start_time
        if elapsed_time >= FRAME_RATE:
            game.update()
            update_grid(map_widget, game.map)
            start_time = time.time()
        await asyncio.sleep(FRAME_RATE)  # Use asyncio's sleep, not time.sleep

# Display the game interface
game_display = VBox([map_widget, up_button, down_button, left_button, right_button, start_button])
display(game_display)


NameError: name 'MultiplayerSnakeGame' is not defined